<a href="https://colab.research.google.com/github/msskx/deep_study/blob/main/%E7%BA%BF%E6%80%A7%E5%9B%9E%E5%BD%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l
import numpy as np
from matplotlib import pyplot as plt

$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$

In [6]:
def sythetic_data(w,b,num_examples):
  X=torch.normal(0,1,(num_examples,len(w)))#生成X,均值为0标准差为1
  y=torch.matmul(X,w)+b#生成y
  y+=torch.normal(0,0.01,y.shape)#增加噪声
  return X,y.reshape((-1,1))

In [8]:
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=sythetic_data(true_w,true_b,1000)

In [33]:
print('features:',features[0],'\nlabel:',labels[0])

features: tensor([-0.5991, -0.8147]) 
label: tensor([5.7883])


In [45]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [49]:
batch_size=10
for X,y in data_iter(batch_size,features,labels):
  print(X,'\n',y)
  break

tensor([[ 0.7134,  0.6269],
        [ 0.3206, -0.5041],
        [-2.1078, -2.0562],
        [ 0.6627, -0.4688],
        [ 0.5774,  0.7444],
        [-1.7222, -0.8365],
        [ 0.4272, -0.3123],
        [-0.7582, -0.7503],
        [-1.3662,  0.9108],
        [-0.7250,  0.4664]]) 
 tensor([[ 3.5158],
        [ 6.5557],
        [ 6.9742],
        [ 7.1182],
        [ 2.8216],
        [ 3.5937],
        [ 6.1151],
        [ 5.2290],
        [-1.6211],
        [ 1.1592]])


## 初始化参数模型

In [50]:
w=torch.normal(0,0.01,size=(2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

## 定义线性模型

In [54]:
def linreg(X,w,b):
  return torch.matmul(X,w)+b

## 定义损失函数

In [55]:
def squared_loss(y_hat,y):
  return(y_hat-y.reshape(y_hat.shape))**2/2

## 定义优化算法

In [56]:
def sgd(params,lr,batch_size):
  with torch.no_grad():
    for param in params:
      param-=lr*param.grad/batch_size
      param.grad.zero_()

## 训练

In [57]:
lr=0.03
num_epochs=3
net=linreg
loss=squared_loss

In [58]:
for epoch in range(num_epochs):
  for X,y in data_iter(batch_size,features,labels):
    l=loss(net(X,w,b),y)

    l.sum().backward()
    sgd([w,b],lr,batch_size)
  with torch.no_grad():
    train_l=loss(net(features,w,b),labels)
    print(f'epoch {epoch+1},loss{float(train_l.mean()):f}')

epoch 1,loss0.034738
epoch 2,loss0.000132
epoch 3,loss0.000052


In [59]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0005, -0.0005], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0010], grad_fn=<RsubBackward1>)


In [60]:
print(f'w:{w}')
print(f'b:{b}')

w:tensor([[ 1.9995],
        [-3.3995]], requires_grad=True)
b:tensor([4.1990], requires_grad=True)
